In [1]:
import custom_functions_pydeseqEnv as cf
import gseapy as gp
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import random
import scanpy as sc
import seaborn as sns

from gseapy.plot import gseaplot
from gseapy import dotplot, barplot
from gseapy.scipalette import SciPalette
from matplotlib.pyplot import rc_context
from matplotlib.backends.backend_pdf import PdfPages
from upsetplot import from_contents, UpSet, plot, generate_counts
from venny4py.venny4py import *

In [2]:
# gp.get_library_name()

In [2]:
### Excluding: 'BioCarta_2016','Panther_2016', 'Reactome_2022', 'Tabula_Muris'
gsetlib = ['GO_Biological_Process_2023'#, 'GO_Cellular_Component_2023',
#            'GO_Molecular_Function_2023', 'KEGG_2019_Mouse', 'WikiPathways_2019_Mouse',
           #'CellMarker_2024', 'Mouse_Gene_Atlas'
          ]

In [3]:
deseqResults_all = pd.read_csv('../output/DEGs/taPVAT_all_comparisons_deseq_all_genes_all.txt', sep = '\t')
deseqResults_all

/tmp/local/47989273/ipykernel_1241406/4039413728.py:1: DtypeWarning: Columns (12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  deseqResults_all = pd.read_csv('../output/DEGs/taPVAT_all_comparisons_deseq_all_genes_all.txt', sep = '\t')


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,celltype,group,group1,group2,subset,sub_id,sub_2,sub_id_2,level
0,3110082J24Rik,32.333176,0.266064,0.368619,0.721786,0.470426,0.721339,3110082J24Rik,whole_tissue,diet,Control,HF,NaN,NaN,NaN,NaN,high
1,4930455H04Rik,4.395147,0.037227,0.380655,0.097796,0.922094,0.973323,4930455H04Rik,whole_tissue,diet,Control,HF,NaN,NaN,NaN,NaN,high
2,6330411D24Rik,14.219259,0.008650,0.313428,0.027599,0.977982,0.992479,6330411D24Rik,whole_tissue,diet,Control,HF,NaN,NaN,NaN,NaN,high
3,A3galt2,19.749140,-0.377105,0.257255,-1.465881,0.142681,0.386271,A3galt2,whole_tissue,diet,Control,HF,NaN,NaN,NaN,NaN,high
4,AABR07000398.1,10208.553315,0.001530,0.206917,0.007396,0.994099,0.997549,AABR07000398.1,whole_tissue,diet,Control,HF,NaN,NaN,NaN,NaN,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5168357,Zzz3,9.884586,-0.326646,0.607374,-0.537801,0.590715,0.999879,Zzz3,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168358,mrpl11,0.620220,-2.044373,2.354850,-0.868154,0.385310,0.999879,mrpl11,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168359,mrpl24,0.681279,-2.036789,1.972784,-1.032444,0.301864,0.999879,mrpl24,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168360,mrpl9,0.675484,-1.412068,1.971169,-0.716361,0.473769,0.999879,mrpl9,ECs_Venous,diet,Control,HF,time,24W,sex,F,low


In [4]:
deseqResults_all.replace(np.nan, 'None', inplace=True)
#deseqResults_all['gene_id'] = deseqResults_all.gene_id.str.upper()

In [7]:
### To only include DEGs in the whole tissue
#celltype_list = ['whole_tissue']
### To restrict to low-level cell types
#celltype_list = ['whole_tissue', 'Adipocytes', 'Endothelial_Cells', 'Fibroblasts', 'Immune_Cells', 'Mesothelial_Cells', 'Neuronal_Cells', 'Pericytes', 'SMCs']
### To restrict to high-level cell types
celltype_list = ['Fibroblasts_Bmper+_Nrxn1+',
 'Adipocytes_2',
 'Macrophages_M2',
 'Adipocytes_Brown',
 'ECs_Venous_Fbn1+',
 'ECs_Lymph',
 'Fibroblasts_Pi16+',
 'ECs_Artery',
 'Adipocytes_3',
 'Pericytes',
 'Macrophages_M2_F13a1+',
 'Mesothelial_Cells',
 'ECs_Cap',
 'Fibroblasts_Bmper+',
 'ECs_Lymph_Piezo2+',
 'Neuronal_Cells',
 'Neutrophils',
 'T_Cells',
 'Doublets',
 'Fibroblasts_Pi16++',
 'ECs_Venous',
 'Macrophages_M1_Cd80+',
 'Dendritic_Cells',
 'B_Cells_Naive',
 'NK_Cells',
 'T_Regs',
 'Monocytes',
 'SMCs']

deseqResults_all = deseqResults_all[deseqResults_all.celltype.isin(celltype_list)]
deseqResults_all

,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,celltype,group,group1,group2,subset,sub_id,sub_2,sub_id_2,level
87824,AABR07000398.1,103.855907,0.163348,0.209676,0.779050,0.435950,0.967643,AABR07000398.1,Pericytes,diet,Control,HF,None,None,None,None,high
87825,AABR07001942.1,3.108596,-0.694927,0.442756,-1.569549,0.116520,0.800895,AABR07001942.1,Pericytes,diet,Control,HF,None,None,None,None,high
87826,AABR07002969.1,2.558075,-0.050935,0.460905,-0.110511,0.912004,0.998868,AABR07002969.1,Pericytes,diet,Control,HF,None,None,None,None,high
87827,AABR07005588.1,3.452993,0.015723,0.380775,0.041292,0.967063,0.998868,AABR07005588.1,Pericytes,diet,Control,HF,None,None,None,None,high
87828,AABR07007032.1,35.788656,-0.157922,0.384404,-0.410823,0.681202,0.995234,AABR07007032.1,Pericytes,diet,Control,HF,None,None,None,None,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5168357,Zzz3,9.884586,-0.326646,0.607374,-0.537801,0.590715,0.999879,Zzz3,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168358,mrpl11,0.620220,-2.044373,2.354850,-0.868154,0.385310,0.999879,mrpl11,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168359,mrpl24,0.681279,-2.036789,1.972784,-1.032444,0.301864,0.999879,mrpl24,ECs_Venous,diet,Control,HF,time,24W,sex,F,low
5168360,mrpl9,0.675484,-1.412068,1.971169,-0.716361,0.473769,0.999879,mrpl9,ECs_Venous,diet,Control,HF,time,24W,sex,F,low


In [11]:
# get significant degs
degs = deseqResults_all[['gene_id','stat', 'pvalue','padj','log2FoldChange', 'celltype', 'group', 'sub_id', 'sub_id_2']]
degs_sig = degs[(degs.padj <= 0.05) & ((degs_sig.log2FoldChange >= 1) | (degs_sig.log2FoldChange <= -1))]
degs_sig

,gene_id,stat,pvalue,padj,log2FoldChange,celltype,group,sub_id,sub_id_2
87967,Acvr1c,4.552292,5.306468e-06,4.596065e-03,2.592054,Pericytes,diet,None,None
88139,Ank3,5.153975,2.550213e-07,3.534085e-04,1.870563,Pericytes,diet,None,None
89165,Csmd1,3.962876,7.405217e-05,3.665054e-02,2.560290,Pericytes,diet,None,None
89950,Fasn,-4.958522,7.103162e-07,8.202969e-04,-2.759684,Pericytes,diet,None,None
90221,Gmpr,-14.643265,1.487506e-48,1.030693e-44,-38.521248,Pericytes,diet,None,None
...,...,...,...,...,...,...,...,...,...
5121159,Sdk1,4.989052,6.067616e-07,1.055158e-03,2.817138,Adipocytes_3,diet,24W,F
5121206,Slc27a1,-3.945579,7.960739e-05,1.977675e-02,-1.990236,Adipocytes_3,diet,24W,F
5131504,Fasn,-5.785267,7.239741e-09,8.790494e-05,-3.904541,ECs_Cap,diet,24W,F
5134763,Ntrk3,5.663756,1.480946e-08,8.990823e-05,3.362349,ECs_Cap,diet,24W,F


In [12]:
degs_sig.celltype.value_counts()

celltype
Adipocytes_Brown             12543
Adipocytes_3                  1542
ECs_Cap                        772
Fibroblasts_Bmper+_Nrxn1+      582
Adipocytes_2                   564
ECs_Artery                     264
Fibroblasts_Pi16+              259
Pericytes                      227
Fibroblasts_Bmper+             193
ECs_Venous                     163
Mesothelial_Cells              135
Fibroblasts_Pi16++             111
ECs_Venous_Fbn1+                70
ECs_Lymph                       69
Macrophages_M2                  68
Macrophages_M2_F13a1+           48
ECs_Lymph_Piezo2+               44
T_Cells                         37
Macrophages_M1_Cd80+            20
B_Cells_Naive                    3
SMCs                             1
Name: count, dtype: int64

## Upset plots

In [ ]:
for name, group in degs_sig[degs_sig.celltype != 'whole_tissue'].groupby(['group', 'sub_id', 'sub_id_2']):
    #print(f'celltype: All \ngroup: {name[0]} \nsub_id: {name[1]} \nsub_id_2: {name[2]}')

    # Group the filtered DataFrame by celltype and collect unique gene_ids and count
    grouped = group.groupby('celltype')['gene_id'].agg(['unique', 'count']).reset_index()

    # Convert the grouped DataFrame to a dictionary
    result_dict = {row['celltype']: {'genes': row['unique'], 'count': row['count']} for _, row in grouped.iterrows()}
    
    # Create a dictionary with cell types as keys and unique gene lists as values
    celltype_gene_dict = {row['celltype']: row['unique'] for _, row in grouped.iterrows()}

    # Convert the dictionary into an UpSet input object
    genes = from_contents(celltype_gene_dict)

    # Plot the UpSet
    fig = plt.figure(figsize=(18, 6))

    # Create the UpSet plot with sorting by count
    upset = UpSet(genes, subset_size='count',show_counts=True, min_subset_size=5, sort_by = 'cardinality', element_size=None)

    upset.plot(fig = fig)
    
    fig.savefig(f"../output/GSEA/UpsetPlot_{name[0]}_{name[1]}_{name[2]}_3000.png", bbox_inches='tight', dpi=600)

## GSEA

In [ ]:
# Initialize an empty list to store GSEA results DataFrames
gsea_results_list = []

for gset in gsetlib:
    print(gset)
    for name, group in deseqResults_all.groupby(['celltype', 'group', 'sub_id', 'sub_id_2']):
        print(f'celltype: {name[0]} \ngroup: {name[1]} \nsub_id: {name[2]} \nsub_id_2: {name[3]}')
                
        #Add a Rank column that takes into account the FDR and fold change
        group['Rank'] = -np.log10(group.padj) * group.log2FoldChange
        
        #Sort by Rank (largest to smallest)
        group = group.sort_values(by='Rank', ascending=False)

        # Prepare the ranking DataFrame
        ranking = group[['gene_id', 'Rank']]

        #Use gseapy.prerank to run GSEA for each cell type
        try:
            pre_res = gp.prerank(rnk=ranking, gene_sets=gset, seed=123, permutation_num=1000)
        
        except Exception as e:
            print(f"Error running GSEA for gene set '{gset}' and cell type '{celltype}'")
            continue 
            
        terms_to_graph = pre_res.res2d.sort_values('FWER p-val').reset_index(drop = True).Term

        gseaplot(rank_metric = pre_res.ranking,
                 term = terms_to_graph[0],
                 **pre_res.results[terms_to_graph[0]])
#         axs = pre_res.plot(terms=terms_to_graph[:5])
        plt.savefig(f'../output/GSEA/taPVAT_{name[0]}_{name[1]}_comparison_{name[2]}_{name[3]}_subset_GSEA_plot_all_genes.pdf',
                    bbox_inches = 'tight', format = 'pdf')
        plt.show()
        
        # Convert the GSEA result to a DataFrame
        gsea_df = pre_res.res2d
        
        # Add a 'celltype' column to store the cell type information
        gsea_df['celltype'] = name[0]
        gsea_df['group'] = name[1]
        gsea_df['sub_id'] = name[2]
        gsea_df['sub_id_2'] = name[3]
        
        gsea_df['gset'] = gset

        # Append the GSEA results DataFrame to the list
        gsea_results_list.append(gsea_df)

# Concatenate all GSEA results DataFrames in the list
gsea_results = pd.concat(gsea_results_list)


In [34]:
color_mapping = dict(zip(celltype_list, cf.bright_palette))

In [ ]:
### making scatter plots of GSEA results
for name, group in gsea_results.groupby(['group', 'sub_id', 'sub_id_2']):
    print(f'celltype: All \ngroup: {name[0]} \nsub_id: {name[1]} \nsub_id_2: {name[2]}')
             
    # Filter GSEA results for NOM p-val, FDR q-val, and FWER p-val < 0.05 and valid 'FDR q-val' values
    filtered_gsea_results = group[
        (group['FWER p-val'] < 0.1)
        #(gsea_results['gset'] == 'GO_Biological_Process_2023')
    ]
    
    ###Save results
    filtered_gsea_results.to_csv(f'../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_filtered_gsea_results.csv')

    # Filter out rows with missing NES or FDR q-val values
    filtered_gsea_results = filtered_gsea_results.dropna(subset=['NES', 'FWER p-val'])

    # Sort the DataFrame by NES in descending order and select the top 15 rows
    filtered_gsea_results = filtered_gsea_results.sort_values(by='FWER p-val', ascending=True).head(15)

    # Print the number of rows in filtered_gsea_results
    print("Number of rows after filtering:", len(filtered_gsea_results))

    # Add a small constant to FDR values to avoid taking the log of zero, used for plotting dot size
    fdr_values = pd.to_numeric(filtered_gsea_results['FWER p-val'] + 1e-10)

    try:
        plt.figure(figsize=(4, 4))

        # Create a scatter plot with adjusted dot size
        scatter = plt.scatter(
            filtered_gsea_results['NES'],  # X-axis: NES
            filtered_gsea_results['Term'],  # Y-axis: GO Term
            s=-np.log10(fdr_values) * 25,  # Adjusted Size: -log10(FDR q-val) with a smaller constant
            c=filtered_gsea_results['celltype'].map(color_mapping),
        )

        # Customize the plot
        plt.xlabel('NES (Normalized Enrichment Score)')
        plt.ylabel('GO Term')
        plt.title('Top GSEA Results (by NES)')

        # Create a legend for the categorical colors
        legend_labels = filtered_gsea_results['celltype'].unique()
        legend = plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10) 
                            for label, color in color_mapping.items()], title='celltype', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.setp(legend.get_title(), fontsize=12)

        # Add grid lines
        plt.grid(True, linestyle='--', alpha=0.7)

        # Adjust the layout to make room for the legend
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_Enriched_GSEA_Results_all_genes.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        # Show the plot
        plt.show()
    
    except Exception as e:
        print(f"Error: No significant GSEA results")



## EnrichR analysis

In [ ]:
sci = SciPalette()
NbDr = sci.create_colormap()

enr_up_results_list = []
enr_dw_results_list = []
enr_up_res2d_list = []
enr_dw_res2d_list = []
enr_res_list = []

for gset in gsetlib:
    print(gset)
    for name, group in degs_sig[degs_sig.celltype != 'whole_tissue'].groupby(['celltype','group', 'sub_id', 'sub_id_2']):
        print(f'celltype: {name[0]}  group: {name[1]}  sub_id: {name[2]}  sub_id_2: {name[3]}')

        degs_up = group[group.log2FoldChange > 0]
        degs_dw = group[group.log2FoldChange < 0]
        
        # Convert degs_up.gene_id to a list-like object
        gene_list_up = degs_up.gene_id.tolist()  # Assuming degs_up.gene_id is a pandas Series
        gene_list_dw = degs_dw.gene_id.tolist()  # Assuming degs_up.gene_id is a pandas Series
        

        # Enricr API
        try:
            enr_up = gp.enrichr(gene_list_up,
                                gene_sets=gset,
                                outdir=None)
        except Exception as e:
            print(f"Error: gene_list_up is empty")  

        try:
            enr_dw = gp.enrichr(gene_list_dw,
                                gene_sets=gset,
                                outdir=None)
        except Exception as e:
            print(f"Error: gene_list_dw is empty") 
            
        # trim (go:...)
        enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]
        enr_dw.res2d.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]
        enr_up.results.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]
        enr_dw.results.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]
        
        enr_up.res2d['celltype'] = name[0]
        enr_up.res2d['group'] = name[1]
        enr_up.res2d['sub_id'] = name[2]
        enr_up.res2d['sub_id_2'] = name[3]
        enr_up.res2d['gset'] = gset
        
        enr_dw.res2d['celltype'] = name[0]
        enr_dw.res2d['group'] = name[1]
        enr_dw.res2d['sub_id'] = name[2]
        enr_dw.res2d['sub_id_2'] = name[3]
        enr_dw.res2d['gset'] = gset
        
        enr_up.results['celltype'] = name[0]
        enr_up.results['group'] = name[1]
        enr_up.results['sub_id'] = name[2]
        enr_up.results['sub_id_2'] = name[3]
        enr_up.results['gset'] = gset
        
        enr_dw.results['celltype'] = name[0]
        enr_dw.results['group'] = name[1]
        enr_dw.results['sub_id'] = name[2]
        enr_dw.results['sub_id_2'] = name[3]
        enr_dw.results['gset'] = gset

        # concat results
        enr_up.res2d['UP_DW'] = "UP"
        enr_dw.res2d['UP_DW'] = "DOWN"
        enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])

        enr_up_res2d_list.append(enr_up.res2d)
        enr_dw_res2d_list.append(enr_dw.res2d)
        enr_up_results_list.append(enr_up.results)
        enr_dw_results_list.append(enr_dw.results)
        enr_res_list.append(enr_res)

enr_up_res2d_all = pd.concat(enr_up_res2d_list)
enr_dw_res2d_all = pd.concat(enr_dw_res2d_list)
enr_up_results_all = pd.concat(enr_up_results_list)
enr_dw_results_all = pd.concat(enr_dw_results_list)

enr_res_all = pd.concat(enr_res_list)

In [ ]:
### plotting enr_up_results
for name, group in enr_up_results_all.groupby(['group', 'sub_id', 'sub_id_2']):
    print(f'celltype: All \ngroup: {name[0]} \nsub_id: {name[1]} \nsub_id_2: {name[2]}')
    
    ### Add try statements so the code doesn't quit running if not enough genes fail below the p-value cut-off
    try:
#         gp.dotplot(group, 
#                    figsize=(3,5), 
#                    title="Up", 
#                    cmap = plt.cm.autumn_r)
        ax = dotplot(group,
                      column="Adjusted P-value",
                      x='celltype', # set x axis, so you could do a multi-sample/library comparsion
                      size=10,
                      top_term=3,
                      figsize=(3,5),
                      title = "",
                      xticklabels_rot=45, # rotate xtick labels
                      show_ring=True, # set to False to revmove outer ring
                      marker='o',
                     )
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_by_celltype_up_EnrichR_Results_dotplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
        
#         # categorical dotplot
        ax = gp.barplot(group,
                      column="Adjusted P-value",
                      group='celltype', # set group, so you could do a multi-sample/library comparsion
                      size=10,
                      top_term=3,
                      figsize=(3,5),
                      color= cf.bright_palette # set colors for group
                      #color = {'KEGG_2021_Human': 'salmon', 'MSigDB_Hallmark_2020':'darkblue'}
                     )
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_by_celltype_up_EnrichR_Results_barplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()

        # to save your figure, make sure that ``ofname`` is not None
        ax = dotplot(group, title='Up',cmap='viridis_r', size=10, figsize=(3,5))
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_all_up_EnrichR_Results_dotplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
    
        # to save your figure, make sure that ``ofname`` is not None
        ax = barplot(group, title='Up', figsize=(4, 5), color='darkred')
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_all_up_EnrichR_Results_barplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()    

    except Exception as e:
        print(f"Error running up dotplot for gene set and cell type")
           
    

In [ ]:
### plotting enr_dw_results
for name, group in enr_dw_results_all.groupby(['group', 'sub_id', 'sub_id_2']):
    print(f'celltype: All \ngroup: {name[0]} \nsub_id: {name[1]} \nsub_id_2: {name[2]}')
    
    ### Add try statements so the code doesn't quit running if not enough genes fail below the p-value cut-off
    try:
#         gp.dotplot(group, 
#                    figsize=(3,5), 
#                    title="Up", 
#                    cmap = plt.cm.autumn_r)
        ax = dotplot(group,
                      column="Adjusted P-value",
                      x='celltype', # set x axis, so you could do a multi-sample/library comparsion
                      size=10,
                      top_term=3,
                      figsize=(3,5),
                      title = "",
                      xticklabels_rot=45, # rotate xtick labels
                      show_ring=True, # set to False to revmove outer ring
                      marker='o',
                     )
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_by_celltype_down_EnrichR_Results_dotplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
        
#         # categorical dotplot
        ax = gp.barplot(group,
                      column="Adjusted P-value",
                      group='celltype', # set group, so you could do a multi-sample/library comparsion
                      size=10,
                      top_term=3,
                      figsize=(3,5),
                      color= cf.bright_palette # set colors for group
                      #color = {'KEGG_2021_Human': 'salmon', 'MSigDB_Hallmark_2020':'darkblue'}
                     )
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_by_celltype_down_EnrichR_Results_barplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()

        # to save your figure, make sure that ``ofname`` is not None
        ax = dotplot(group, title='Up',cmap='viridis_r', size=10, figsize=(3,5))
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_all_down_EnrichR_Results_dotplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
    
        # to save your figure, make sure that ``ofname`` is not None
        ax = barplot(group, title='Up', figsize=(4, 5), color='darkred')
        plt.savefig(f"../output/GSEA/taPVAT_All_{name[0]}_comparison_{name[1]}_{name[2]}_subset_all_down_EnrichR_Results_barplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()    

    except Exception as e:
        print(f"Error running up dotplot for gene set and cell type")
           
    

In [ ]:
### enr_up_dw_results
for name, group in enr_res_all.groupby(['celltype','group', 'sub_id', 'sub_id_2']):
    print(f'celltype: {name[0]} \ngroup: {name[1]} \nsub_id: {name[2]} \nsub_id_2: {name[3]}')
        
    try:
        # display multi-datasets
        ax = gp.dotplot(group, figsize=(3,5),
                        x='UP_DW',
                        x_order = ["UP","DOWN"],
                        title="GO_BP",
                        cmap = NbDr.reversed(),
                        size=6,
                        show_ring=True)
        ax.set_xlabel("")
        plt.savefig(f"../output/GSEA/taPVAT_{name[0]}_{name[1]}_comparison_{name[2]}_{name[3]}_subset_up_down_EnrichR_Results_dotplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()

        ax = gp.barplot(group, figsize=(3,5),
                    group ='UP_DW',
                    title ="GO_BP",
                    color = ['b','r'])
        plt.savefig(f"../output/GSEA/taPVAT_{name[0]}_{name[1]}_comparison_{name[2]}_{name[3]}_subset_up_down_EnrichR_Results_barplot.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
    except Exception as e:
        print(f"Error running up/down dotplot for gene set and cell type")

In [ ]:
###Networkx graph: up
for name, group in enr_up_res2d_all.groupby(['celltype','group', 'sub_id', 'sub_id_2']):
    print(f'celltype: {name[0]} \ngroup: {name[1]} \nsub_id: {name[2]} \nsub_id_2: {name[3]}')
    
    try:
        nodes, edges = gp.enrichment_map(group, cutoff = 0.05)
        # build graph
        G = nx.from_pandas_edgelist(edges,
                                    source='src_idx',
                                    target='targ_idx',
                                    edge_attr=['jaccard_coef', 'overlap_coef', 'overlap_genes'])

        fig, ax = plt.subplots(figsize=(8, 8))

        # init node cooridnates
        pos=nx.layout.spiral_layout(G)
        #node_size = nx.get_node_attributes()
        # draw node
        nx.draw_networkx_nodes(G,
                               pos=pos,
                               cmap=plt.cm.RdYlBu,
                               node_color=list(nodes['Combined Score']),
                               node_size=list(nodes.Hits_ratio *1000)
                              )
        # draw node label
        nx.draw_networkx_labels(G,
                                pos=pos,
                                labels=nodes.Term.to_dict(),
                                font_size=10
                               )
        # draw edge
        edge_weight = nx.get_edge_attributes(G, 'jaccard_coef').values()
        nx.draw_networkx_edges(G,
                               pos=pos,
                               width=list(map(lambda x: x*10, edge_weight)),
                               edge_color='#CDDBD4')
        
        plt.savefig(f"../output/GSEA/taPVAT_{name[0]}_{name[1]}_comparison_{name[2]}_{name[3]}_subset_up_EnrichR_Results_network.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
    except Exception as e:
        print(f"Error: No enrich terms when cutoff = 0.05")

In [ ]:
###Networkx graph: down
for name, group in enr_dw_res2d_all.groupby(['celltype','group', 'sub_id', 'sub_id_2']):
    print(f'celltype: {name[0]} \ngroup: {name[1]} \nsub_id: {name[2]} \nsub_id_2: {name[3]}')

    try:
        nodes, edges = gp.enrichment_map(group, cutoff = 0.05)
        # build graph
        G = nx.from_pandas_edgelist(edges,
                                    source='src_idx',
                                    target='targ_idx',
                                    edge_attr=['jaccard_coef', 'overlap_coef', 'overlap_genes'])

        fig, ax = plt.subplots(figsize=(8, 8))

        # init node cooridnates
        pos=nx.layout.spiral_layout(G)
        #node_size = nx.get_node_attributes()
        # draw node
        nx.draw_networkx_nodes(G,
                               pos=pos,
                               cmap=plt.cm.RdYlBu,
                               node_color=list(nodes['Combined Score']),
                               node_size=list(nodes.Hits_ratio *1000)
                              )
        # draw node label
        nx.draw_networkx_labels(G,
                                pos=pos,
                                labels=nodes.Term.to_dict(),
                                font_size=10
                               )
        # draw edge
        edge_weight = nx.get_edge_attributes(G, 'jaccard_coef').values()
        nx.draw_networkx_edges(G,
                               pos=pos,
                               width=list(map(lambda x: x*10, edge_weight)),
                               edge_color='#CDDBD4')
        
        plt.savefig(f"../output/GSEA/taPVAT_{name[0]}_{name[1]}_comparison_{name[2]}_{name[3]}_subset_down_EnrichR_Results_network.pdf", 
                    bbox_inches='tight', format = 'pdf', dpi=600)
        plt.show()
    except Exception as e:
        print(f"Error: No enrich terms when cutoff = 0.05")